# Create satisfactory db

Using json from Tomáš Blatný [Satisfacroy Tools](https://github.com/greeny/SatisfactoryTools/tree/dev/data)
Dowload data.json,
Rename it satisfactory.json

In [1]:
import json
import os

from sqlalchemy import create_engine, Table, Column, Integer, String, Boolean, MetaData, Float

In [2]:
with open('satisfactory.json') as datas:
    datas = json.load(datas)
    
for i in datas:
    print(i)
    
recipes_src = datas['recipes']
items_src = datas['items']
schematics_src = datas['schematics']
generators_src = datas['generators']
resources_src = datas['resources']
miners_src = datas['miners']
buildings_src = datas['buildings']

for i in items_src.items():
    print(i)
    break
    
for i in recipes_src.items():
    print(i)
    break
    
for i in recipes_src.values():
    if i['forBuilding']:
        print(i)
        break
    
for i in buildings_src.items():
    print(i)
    break

recipes
items
schematics
generators
resources
miners
buildings
('BP_EqDescZipLine_C', {'slug': 'zipline', 'className': 'BP_EqDescZipLine_C', 'name': 'Zipline', 'sinkPoints': 5284, 'description': 'Slot: Hand\n\nProvides faster traversal in factories by allowing Pioneers to zip along Power lines.\nActivate the Zipline and aim at a nearby Power Line to connect to it.', 'stackSize': 1, 'energyValue': 0, 'radioactiveDecay': 0, 'liquid': False, 'fluidColor': {'r': 0, 'g': 0, 'b': 0, 'a': 0}})
('Recipe_AILimiter_C', {'slug': 'ai-limiter', 'name': 'AI Limiter', 'className': 'Recipe_AILimiter_C', 'alternate': False, 'time': 12, 'manualTimeMultiplier': 1, 'ingredients': [{'item': 'Desc_CopperSheet_C', 'amount': 5}, {'item': 'Desc_HighSpeedWire_C', 'amount': 20}], 'forBuilding': False, 'inMachine': True, 'inHand': True, 'inWorkshop': False, 'products': [{'item': 'Desc_CircuitBoardHighSpeed_C', 'amount': 1}], 'producedIn': ['Desc_AssemblerMk1_C']})
{'slug': 'assembler', 'name': 'Assembler', 'class

In [3]:
satisfactory_db = "satisfactory.db"

if os.path.exists(satisfactory_db):
    os.unlink(satisfactory_db)

In [4]:
engine = create_engine(f'sqlite:///{satisfactory_db}')

In [5]:
meta = MetaData()

recipes = Table(
    'recipes', meta, 
    Column('id', Integer, primary_key = True), 
    Column('slug', String), 
    Column('name', String), 
    Column('className', String), 
    Column('alternate', Boolean), 
    Column('time', Float), 
    Column('manualTimeMultiplier', Integer), 
    Column('forBuilding', Boolean), 
    Column('inMachine', Boolean), 
    Column('inHand', Boolean), 
    Column('inWorkshop', Boolean),
    Column('producedIn', String)
)

items = Table(
    'items', meta,
    Column('id', Integer, primary_key = True),
    Column('slug', String), 
    Column('name', String), 
    Column('className', String), 
    Column('sinkPoints', Integer),
    Column('description', String),
    Column('stackSize', Integer),
    Column('energyValue', Integer),
    Column('radioactiveDecay', Integer),
    Column('liquid', Boolean)
)

recipe_ingredients = Table(
    'recipe_ingredients', meta,
    Column('id', Integer, primary_key = True),
    Column('recipe', Integer),
    Column('item', Integer),
    Column('amount', Integer)
)

recipe_products = Table(
    'recipe_products', meta,
    Column('id', Integer, primary_key = True),
    Column('recipe', Integer),
    Column('item', Integer),
    Column('amount', Integer)
)

buildings = Table(
    'buildings', meta,
    Column('id', Integer, primary_key = True),
    Column('slug', String), 
    Column('name', String), 
    Column('description', String), 
    # Not categories
    Column('buildMenuPriority', Integer),
    Column('className', String),
    # Not metadata
    # Not size but: (0 for now in source !) 
    Column('width', Integer),
    Column('length', Integer),
    Column('height', Integer),
    Column('recipe', Integer)
)

meta.create_all(engine)

In [6]:
def add_item(it):
    if 'fluidColor' in it: del i['fluidColor']
    ins = items.insert().values(it)
    return engine.execute(ins)

for i in items_src.values():
    add_item(i)

add_item({
    "slug": "plutonium-waste",
    "className": "Desc_PlutoniumWaste_C",
    "name": "Plutonium Waste",
    "sinkPoints": 0,
    "description": "The by-product of consuming Plutonium Fuel Rods in the Nuclear Power Plant.\n\nCaution: HIGHLY Radioactive.",
    "stackSize": 500,
    "energyValue": 0,
    "radioactiveDecay": 20,
    "liquid": False
})

add_item({
    'slug': 'electricity',
    'className': 'BP_Electricity_C', 
    'name': 'electricity',
    'sinkPoints': 0,
    'description': 'to Power everything',
    'stackSize': 0,
    'energyValue': 1,
    'radioactiveDecay': 0,
    'liquid': False
})

In [7]:
for b in buildings_src.values():
    if 'categories' in b:
        del b['categories']
    if 'metadata' in b:
        del b['metadata']
    if 'size' in b:
        for desc in b['size']:
            b[desc] = b['size'][desc]
        del b['size']
    
    ins = buildings.insert().values(b)
    
    engine.execute(ins)

In [8]:
for r in recipes_src.values():
    
    c = dict(r)
    del c['ingredients']
    del c['products']
    if len(c['producedIn']) > 0:
        if len(c['producedIn']) > 1:
            print(c['slug'])
        c['producedIn'] = c['producedIn'][0]
    else:
        c['producedIn'] = None
    
    ins = recipes.insert().values(c)
    a = engine.execute(ins)
    key = a.inserted_primary_key[0]
    for ing in r['ingredients']:
        it = engine.execute(items.select().where(items.c.className == ing['item'])).fetchone()
        if it is not None:
            ing_id = it[0]
            ins = recipe_ingredients.insert().values(recipe = key, item = ing_id, amount = ing['amount'])
            engine.execute(ins)
        
    if r['forBuilding']:
        for prod in r['products']:
            assert prod['amount'] == 1
            ins = (buildings.update()
                   .where(buildings.c.className == prod['item'])
                   .values(recipe = key))
            engine.execute(ins)
    else:
        for prod in r['products']:
            it = engine.execute(items.select().where(items.c.className == prod['item'])).fetchone()
            if it is not None:
                prod_id = it[0]
                ins = recipe_products.insert().values(recipe = key, item = prod_id, amount = prod['amount'])
                engine.execute(ins)
            
    #if r['forBuilding']:
    #    print(f'{key}: {r}')
        


In [9]:
# Fake recipe for electricity

special_case = {
    'Build_GeneratorNuclear_C': {
        'Desc_NuclearFuelRod_C': ("Desc_NuclearWaste_C", 50),
        'Desc_PlutoniumFuelRod_C': ("Desc_PlutoniumWaste_C", 10)
    }
}

for gen in generators_src.values():
    print(gen)
    break
        
for gen in generators_src.values():
    desc_class = 'Desc_' + gen['className'][6:]
    desc = buildings_src[desc_class]
    gen_slug = desc['slug']
    gen_name = desc['name']
    for fuel in gen['fuel']:
        desc_fuel = items_src[fuel]
        if desc_fuel['energyValue'] == 0:
            continue
        time = desc_fuel['energyValue'] / gen['powerProduction']
        fake_recipe = {
            "slug": f"{desc_fuel['slug']}-{desc['slug']}",
            "name": f"{desc_fuel['name']} in {desc['name']}",
            "className": "FakeClassName",
            "alternate": False,
            "time": time,
            "manualTimeMultiplier": 1,
            "forBuilding": False,
            "inMachine": True,
            "inHand": False,
            "inWorkshop": False,
            "producedIn": desc_class
        }
        ins = recipes.insert().values(fake_recipe)
        a = engine.execute(ins)
        key = a.inserted_primary_key[0]
        
        it = engine.execute(items.select().where(items.c.className == fuel)).fetchone()
        assert(it is not None)
        ing_id = it[0]
        ins = recipe_ingredients.insert().values(recipe = key, item = ing_id, amount = 1)
        engine.execute(ins)
        
        it = engine.execute(items.select().where(items.c.className == 'BP_Electricity_C')).fetchone()
        assert(it is not None)
        prod_id = it[0]
        ins = recipe_products.insert().values(recipe = key, item = prod_id, amount = desc_fuel['energyValue']/60)
        engine.execute(ins)
        

        

{'className': 'Build_GeneratorBiomass_C', 'fuel': ['Desc_Biofuel_C', 'Desc_GenericBiomass_C', 'Desc_Leaves_C', 'Desc_Wood_C', 'Desc_LiquidBiofuel_C', 'Desc_PackagedBiofuel_C', 'Desc_PackagedAlumina_C', 'Desc_Fabric_C', 'Desc_Mycelia_C', 'Desc_PackagedSulfuricAcid_C', 'Desc_PackagedNitrogenGas_C', 'Desc_PackagedNitricAcid_C', 'Desc_HogParts_C', 'Desc_SpitterParts_C', 'Desc_FlowerPetals_C'], 'powerProduction': 30, 'powerProductionExponent': 1.3, 'waterToPowerRatio': 0}


In [10]:
recipes_src["Recipe_IronRod_C"]

{'slug': 'iron-rod',
 'name': 'Iron Rod',
 'className': 'Recipe_IronRod_C',
 'alternate': False,
 'time': 4,
 'manualTimeMultiplier': 0.5,
 'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
 'forBuilding': False,
 'inMachine': True,
 'inHand': True,
 'inWorkshop': False,
 'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
 'producedIn': ['Desc_ConstructorMk1_C']}

In [39]:
def make_part(name, stackSize):
    return {
        'slug': name.lower().replace(' ','-'),
        'className': 'Desc_' + name.replace(' ','') + '_C',
        'name': name,
        'sinkPoints': 0,
        'description': 'no description',
        'stackSize': stackSize,
        'energyValue': 0,
        'radioactiveDecay': 0,
        'liquid': False,
        'fluidColor': {'r': 0, 'g': 0, 'b': 0, 'a': 0}}

def make_recipe(name, time):
    return {
        'slug': name.lower().replace(' ','-'),
        'name': name,
        'className': 'Desc_' + name.replace(' ','') + '_C',
        'alternate': False,
        'time': time,
        'manualTimeMultiplier': 1 ,
        'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
        'forBuilding': False,
        'inMachine': True,
        'inHand': True,
        'inWorkshop': False,
        'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
        'producedIn': ['Desc_ConstructorMk1_C']}

In [40]:
xmas_parts = [ make_part(name, stack) for name, stack in
    [
        ('FICSMAS Gift', 500), ('Candy Cane', 500), ('Actual Snow', 500),
        ('Red FICSMAS Ornament', 500), ('Blue FICSMAS Ornament', 500),
        ('Copper FICSMAS Ornament', 200), ('Iron FICSMAS Ornament', 200),
        ('FICSMAS Ornament Bundle', 100),
        ('FICSMAS Bow', 500),
        ('FICSMAS Tree Branch', 500),
        ('FICSMAS Wonder Star', 50),
        ('FICSMAS Decoration', 100),
        ('Snowball', 500)
    ]]

In [41]:
xmas_recipes = [
    {'slug': 'iron-rod',
 'name': 'Iron Rod',
 'className': 'Recipe_IronRod_C',
 'alternate': False,
 'time': 4,
 'manualTimeMultiplier': 0.5,
 'ingredients': [{'item': 'Desc_IronIngot_C', 'amount': 1}],
 'forBuilding': False,
 'inMachine': True,
 'inHand': True,
 'inWorkshop': False,
 'products': [{'item': 'Desc_IronRod_C', 'amount': 1}],
 'producedIn': ['Desc_ConstructorMk1_C']}
]

Correction